In [1]:
import numpy as np
import pandas as pd
from astroquery.jplhorizons import Horizons
from astropy.time import Time
from astropy import units as u
from astropy.coordinates import SkyCoord
import matplotlib.pyplot as plt
import time
from astropy.coordinates import GeocentricMeanEcliptic
from astropy.coordinates import HeliocentricMeanEcliptic
from astropy.coordinates import HeliocentricTrueEcliptic
from astropy.coordinates import GeocentricTrueEcliptic
from astropy.coordinates import BarycentricTrueEcliptic
from astropy.coordinates import BarycentricMeanEcliptic
from astropy import coordinates 
import astropy

#latlon = pd.read_csv('NewLempo_LatLon.csv', header=0)

latlon = pd.read_csv('NewEris_LatLon.csv', header=0)

date = latlon['time']
dateList = []
for i in date:
    jd = Time(i,format='jd')
    dateList.append(jd)
        
primary = Horizons(id='Lempo',location=None,epochs=dateList)
    
dist = primary.vectors()['range']

Lat_Prim = np.array(latlon['Lat_Prim'])
Long_Prim = np.array(latlon['Long_Prim'])
DLat_Paha = np.array(latlon['DeltaLat_Paha'])
DLong_Paha = np.array(latlon['DeltaLong_Paha'])
#DLat_Hiisi = np.array(latlon['DeltaLat_Hiisi'])
#DLong_Hiisi = np.array(latlon['DeltaLong_Hiisi'])

Lat2 = Lat_Prim + DLat_Paha/3600
Long2 = DLong_Paha/3600/np.cos(Lat_Prim*u.degree)+Long_Prim
#Lat3 = Lat_Prim + DLat_Hiisi/3600
#Long3 = DLong_Hiisi/3600/np.cos(Lat_Prim*u.degree)+Long_Prim

primEcl = SkyCoord(lat=np.array(Lat_Prim)*u.degree, lon=np.array(Long_Prim)*u.degree, frame='geocentricmeanecliptic',distance = dist,unit=(u.deg,u.deg))

PahaEcl = SkyCoord(lat=np.array(Lat2)*u.degree, lon=np.array(Long2)*u.degree, frame='geocentricmeanecliptic',distance = dist,unit=(u.deg,u.deg))
#HiisiEcl = SkyCoord(lat=np.array(Lat3)*u.degree, lon=np.array(Long3)*u.degree, frame='heliocentricmeanecliptic',distance = dist,unit=(u.deg,u.deg))

Paha = PahaEcl.transform_to('icrs')
#Hiisi = HiisiEcl.transform_to('icrs')
prim = primEcl.transform_to('icrs')

RA1 = prim.ra.degree
DEC1 = prim.dec.degree
RA2 = Paha.ra.degree
DEC2 = Paha.dec.degree
#RA3 = Hiisi.ra.degree
#DEC3 = Hiisi.dec.degree

#print(RA2)

newdf = pd.DataFrame()
newdf['time'] = date
newdf['RA_Prim'] = RA1
newdf['DEC_Prim'] = DEC1
newdf['DRA2'] = (RA2-RA1)*3600*np.cos(DEC1*u.degree)
newdf['DDEC2'] = (DEC2-DEC1)*3600
#newdf['DRA3'] = (RA3-RA1)*3600*np.cos(DEC1*u.degree)
#newdf['DDEC3'] = (DEC3-DEC1)*3600

print(newdf)


            time    RA_Prim  DEC_Prim      DRA2     DDEC2
0   2.453623e+06  24.306494 -5.473827 -0.520085  0.080013
1   2.453707e+06  24.334919 -5.415299  0.206034 -0.338055
2   2.453968e+06  24.423085 -5.233684 -0.148025  0.348057
3   2.453969e+06  24.423415 -5.233004 -0.360060  0.260042
4   2.453978e+06  24.426466 -5.226733  0.488081 -0.200032
5   2.453978e+06  24.426556 -5.226523  0.504083 -0.167027
6   2.453978e+06  24.426776 -5.226083  0.520086 -0.072011
7   2.454325e+06  24.544468 -4.983699  0.484750 -0.178129
8   2.457051e+06  25.483633 -3.063829 -0.350089 -0.226068
9   2.457051e+06  25.483663 -3.063779 -0.341447 -0.229169
10  2.457054e+06  25.484754 -3.061539  0.287389 -0.328135
11  2.457054e+06  25.484774 -3.061489  0.297690 -0.323734
12  2.458119e+06  25.848202 -2.308348 -0.128202  0.363161
13  2.458119e+06  25.848262 -2.308208 -0.171809  0.362541
14  2.458121e+06  25.848872 -2.306938 -0.454697  0.226388
15  2.458127e+06  25.850913 -2.302707  0.145545 -0.366561
16  2.458128e+

In [2]:
radec = pd.read_csv('BenecchiData_NotWrong.csv', header=0)

#latlon = pd.read_csv('Lempo_obs_df_old.csv', header=0)

latlon = pd.DataFrame(columns = ['time'])
    
date = radec['time']
dateList = []
for i in date:
    jd = Time(i,format='jd')
    dateList.append(jd)
        
    #Get the Horizons data for the object at the times it was observed
primary = Horizons(id='Lempo',location=None,epochs=dateList)
    
latlon['time'] = radec['time']-primary.vectors()['lighttime']
    
    #Pull all data from csv file
RA_Prim = np.array(radec['RA-Primary'])
DEC_Prim = np.array(radec['DEC-Primary'])
    
deltaRA_Paha = np.array(radec['Delta-RA_Paha'])
deltaDEC_Paha = np.array(radec['Delta-DEC_Paha'])
deltaRA_Hiisi = np.array(radec['Delta-RA_Hiisi'])
deltaDEC_Hiisi = np.array(radec['Delta-DEC_Hiisi'])
    
    #Convert the deltas back to degrees from arcseconds

RA_Paha = RA_Prim+deltaRA_Paha/3600/np.cos(DEC_Prim*u.degree)
DEC_Paha = DEC_Prim + deltaDEC_Paha/3600
      
RA_Hiisi = RA_Prim+deltaRA_Hiisi/3600/np.cos(DEC_Prim*u.degree)
DEC_Hiisi = DEC_Prim + deltaDEC_Hiisi/3600
  
radf = pd.DataFrame()
radf['RA-Prim'] = RA_Prim
radf['DEC-Prim'] = DEC_Prim

radf['DRA_Paha'] = deltaRA_Paha
radf['DDEC_Paha'] = deltaDEC_Paha
radf['DRA_Hiisi'] = deltaRA_Hiisi
radf['DDEC_Hiisi'] = deltaDEC_Hiisi
#print(radf)
    
    #Essentially we define where the object is in our RA/DEC coordinate system. ICRS is the system our coordinates are in.
dist = primary.vectors()['range']
PahaC = SkyCoord(ra=RA_Paha*u.degree, dec=DEC_Paha*u.degree, frame='icrs',distance = dist,unit=(u.deg,u.deg))
HiisiC = SkyCoord(ra=RA_Hiisi*u.degree, dec=DEC_Hiisi*u.degree, frame='icrs',distance = dist,unit=(u.deg,u.deg))
        
primC = SkyCoord(ra=RA_Prim*u.degree, dec=DEC_Prim*u.degree, frame='icrs',distance = dist,unit=(u.deg,u.deg))
    
    #Transform your icrs frame to a J2000 latitude and longitude ecliptic frame
PahaEcl = PahaC.transform_to(HeliocentricMeanEcliptic(equinox='J2000'))
HiisiEcl = HiisiC.transform_to(HeliocentricMeanEcliptic(equinox='J2000'))
    
primEcl = primC.transform_to(HeliocentricMeanEcliptic(equinox='J2000'))
    
Lat_Prim = primEcl.lat.degree
Long_Prim = primEcl.lon.degree
    
Lat_Paha = PahaEcl.lat.degree
Long_Paha = PahaEcl.lon.degree
    
Lat_Hiisi = HiisiEcl.lat.degree
Long_Hiisi = HiisiEcl.lon.degree

DeltaLat_Paha = (Lat_Paha-Lat_Prim)*3600
DeltaLong_Paha = (Long_Paha-Long_Prim)*np.cos(Lat_Prim*u.degree)*3600
    
DeltaLat_Hiisi = (Lat_Hiisi-Lat_Prim)*3600
DeltaLong_Hiisi = (Long_Hiisi-Long_Prim)*np.cos(Lat_Prim*u.degree)*3600

Lat_Paha = Lat_Prim + DeltaLat_Paha/3600
Long_Paha = DeltaLong_Paha/3600/np.cos(Lat_Prim*u.degree)+Long_Prim
Lat_Hiisi = Lat_Prim + DeltaLat_Hiisi/3600
Long_Hiisi = DeltaLong_Hiisi/3600/np.cos(Lat_Prim*u.degree)+Long_Prim


print(Lat_Prim)
print(np.cos(Lat_Prim*u.degree))
print(DeltaLong_Paha/3600/np.cos(Lat_Prim*u.degree))

latlon['Lat_Prim'] = Lat_Prim
latlon['Long_Prim'] = Long_Prim
latlon['DLat_Paha'] = DeltaLat_Paha
latlon['DLong_Paha'] = DeltaLong_Paha
latlon['DLat_Hiisi'] = DeltaLat_Hiisi
latlon['DLong_Hiisi'] = DeltaLong_Hiisi

#print(Long_Paha)
#print(latlon)

primEcl = SkyCoord(lat=Lat_Prim*u.degree, lon=Long_Prim*u.degree, frame='geocentricmeanecliptic',distance = dist,unit=(u.deg,u.deg))

PahaEcl = SkyCoord(lat=Lat_Paha*u.degree, lon=Long_Paha*u.degree, frame='geocentricmeanecliptic',distance = dist,unit=(u.deg,u.deg))
HiisiEcl = SkyCoord(lat=Lat_Hiisi*u.degree, lon=Long_Hiisi*u.degree, frame='geocentricmeanecliptic',distance = dist,unit=(u.deg,u.deg))

Paha = PahaEcl.transform_to('icrs')
Hiisi = HiisiEcl.transform_to('icrs')
prim = primEcl.transform_to('icrs')

RA1 = prim.ra.degree
DEC1 = prim.dec.degree
RA2 = Paha.ra.degree
DEC2 = Paha.dec.degree
RA3 = Hiisi.ra.degree
DEC3 = Hiisi.dec.degree

#print(RA2)
newdf = pd.DataFrame()
newdf['time'] = date
newdf['RA_Prim'] = RA1
newdf['DEC_Prim'] = DEC1
newdf['DRA2'] = (RA2-RA1)*3600*np.cos(DEC1*u.degree)
newdf['DDEC2'] = (DEC2-DEC1)*3600
newdf['DRA3'] = (RA3-RA1)*3600*np.cos(DEC1*u.degree)
newdf['DDEC3'] = (DEC3-DEC1)*3600


print(newdf)

NameError: name 'pd' is not defined

In [1]:
import numpy as np
import pandas as pd
from astroquery.jplhorizons import Horizons
from astropy.time import Time
from astropy import units as u
from astropy.coordinates import SkyCoord
import matplotlib.pyplot as plt
import time
from astropy.coordinates import GeocentricMeanEcliptic
from astropy.coordinates import HeliocentricMeanEcliptic
from astropy.coordinates import HeliocentricTrueEcliptic
from astropy.coordinates import GeocentricTrueEcliptic
from astropy.coordinates import BarycentricTrueEcliptic
from astropy.coordinates import BarycentricMeanEcliptic
from astropy import coordinates 
import astropy

#latlon = pd.read_csv('NewLempo_LatLon.csv', header=0)

latlon = pd.read_csv("Eris_positions.csv",header=0)

date = latlon['time']
dateList = []
for i in date:
    jd = Time(i,format='jd')
    dateList.append(jd)
        
primary = Horizons(id='Eris',location=None,epochs=dateList)
    
dist = primary.vectors()['range']

Lat_Prim = np.array(latlon['Lat_Prim'])
Long_Prim = np.array(latlon['Long_Prim'])
DLat_Paha = np.array(latlon['DeltaLat_Hercules'])
DLong_Paha = np.array(latlon['DeltaLong_Hercules'])
#DLat_Hiisi = np.array(latlon['DeltaLat_Hiisi'])
#DLong_Hiisi = np.array(latlon['DeltaLong_Hiisi'])

Lat2 = Lat_Prim + DLat_Paha/3600
Long2 = DLong_Paha/3600/np.cos(Lat_Prim*u.degree)+Long_Prim
#Lat3 = Lat_Prim + DLat_Hiisi/3600
#Long3 = DLong_Hiisi/3600/np.cos(Lat_Prim*u.degree)+Long_Prim

primEcl = SkyCoord(lat=np.array(Lat_Prim)*u.degree, lon=np.array(Long_Prim)*u.degree, frame='geocentricmeanecliptic',distance = dist,unit=(u.deg,u.deg))

PahaEcl = SkyCoord(lat=np.array(Lat2)*u.degree, lon=np.array(Long2)*u.degree, frame='geocentricmeanecliptic',distance = dist,unit=(u.deg,u.deg))
#HiisiEcl = SkyCoord(lat=np.array(Lat3)*u.degree, lon=np.array(Long3)*u.degree, frame='heliocentricmeanecliptic',distance = dist,unit=(u.deg,u.deg))

Paha = PahaEcl.transform_to('icrs')
#Hiisi = HiisiEcl.transform_to('icrs')
prim = primEcl.transform_to('icrs')

RA1 = prim.ra.degree
DEC1 = prim.dec.degree
RA2 = Paha.ra.degree
DEC2 = Paha.dec.degree
#RA3 = Hiisi.ra.degree
#DEC3 = Hiisi.dec.degree

#print(RA2)

newdf = pd.DataFrame()
newdf['time'] = date
newdf['RA_Prim'] = RA1
newdf['DEC_Prim'] = DEC1
newdf['DRA2'] = (RA2-RA1)*3600*np.cos(DEC1*u.degree)
newdf['DDEC2'] = (DEC2-DEC1)*3600
#newdf['DRA3'] = (RA3-RA1)*3600*np.cos(DEC1*u.degree)
#newdf['DDEC3'] = (DEC3-DEC1)*3600

print(newdf)


          time    RA_Prim  DEC_Prim      DRA2     DDEC2
0  2453977.525  25.372132 -4.932013  0.213186 -0.021775
1  2457051.110  25.465157 -3.003452  0.220460 -0.041071
2  2457051.179  25.465314 -3.003188  0.214014 -0.011413
3  2457054.359  25.473032 -2.990929  0.210791  0.003408
4  2458128.364  25.823514 -2.291278  0.198379  0.035632
